In [1]:
from sparq.tools.python_repl.python_repl_tool import python_repl_tool
from sparq.utils.helpers import get_llm
from sparq.settings import Settings

from pathlib import Path
import os

from langgraph.prebuilt import create_react_agent
from langchain.tools import tool

In [7]:
@tool
def my_tool_function(input: str) -> str:
    """
    A simple tool function that processes the input string and returns a formatted string.
    """
    return f"Processed input: {input}"

In [6]:
settings = Settings()
settings._load_env_variables(Path("../.env"))


if os.getenv("GEMINI_API_KEY"):
    model = "gemini-2.0-flash"
    provider = "google_genai"
    llm = get_llm(model=model, provider=provider)
else:
    raise ValueError("GEMINI_API_KEY is not set in the environment variables.")

In [8]:
try:
    agent = create_react_agent(
        model=llm,
        tools=[my_tool_function, python_repl_tool]
    )
    print("Agent created successfully with python REPL tool")
except Exception as e:
    print(f"Error creating agent: {e}")

Agent created successfully with python REPL tool


In [9]:
try:
    prompt = "Use python to add two numbers: 3.9345 and 5.4345"
    input = {"messages": [{"role": "user", "content": prompt}]}
    output = agent.invoke(input)
    print(f"Agent output: {output}")

    print("Type(output):", type(output))
    print("repr(output):", repr(output))
except Exception as e:
    print(f"Error running agent: {e}")

Agent output: {'messages': [HumanMessage(content='Use python to add two numbers: 3.9345 and 5.4345', additional_kwargs={}, response_metadata={}, id='a62126e9-9970-49ec-b8cc-cbb4e30e4ca7'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_repl_tool', 'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f21e11f9-98fa-46c6-b779-52d59cdbeb88-0', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(3.9345 + 5.4345)'}, 'id': '9de84b79-44ef-489c-a20c-9c512a2babb3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 23, 'total_tokens': 200, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='✓ Code executed successfully.\nOutput:\n9.369', name='python_repl_tool', id='7ad7fae7-b473-4da8-941a-6149536bb136', tool_call_id='9de84b79-44ef-489c-a20c-9c512a

In [6]:
3.9345 + 5.4345

9.369

In [10]:
print("information on the output from agent.invoke()")
print("Type(output):", type(output))
print("repr(output):", repr(output))
print("len(output):", len(output))
print("output.keys():", output.keys())
print()
print("type(output['messages']):", type(output["messages"]))
print("len(output['messages']):", len(output["messages"]))
print(output['messages'])
print()
print("type(output['messages'][0]):", type(output["messages"][0]))
print("output['messages'][0].pretty_print():", output['messages'][0].pretty_print())
print()
print(output['messages'][1].pretty_print())

information on the output from agent.invoke()
Type(output): <class 'langgraph.pregel.io.AddableValuesDict'>
repr(output): {'messages': [HumanMessage(content='Use python to add two numbers: 3.9345 and 5.4345', additional_kwargs={}, response_metadata={}, id='a62126e9-9970-49ec-b8cc-cbb4e30e4ca7'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'python_repl_tool', 'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f21e11f9-98fa-46c6-b779-52d59cdbeb88-0', tool_calls=[{'name': 'python_repl_tool', 'args': {'code': 'print(3.9345 + 5.4345)'}, 'id': '9de84b79-44ef-489c-a20c-9c512a2babb3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 23, 'total_tokens': 200, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='✓ Code executed successfully.\nOutput:\n9.369', name

In [11]:
output['messages'][1].additional_kwargs

{'function_call': {'name': 'python_repl_tool',
  'arguments': '{"code": "print(3.9345 + 5.4345)"}'}}

In [12]:
output['messages'][1].additional_kwargs['function_call']['arguments']

'{"code": "print(3.9345 + 5.4345)"}'

In [9]:
import json
print(json.loads(output['messages'][1].additional_kwargs['function_call']['arguments']))

# get the code
code = json.loads(output['messages'][1].additional_kwargs['function_call']['arguments'])['code']
print(code)

{'code': 'print(3.9345 + 5.4345)'}
print(3.9345 + 5.4345)


In [13]:
# Diagnostic: Test basic multiprocessing
import multiprocessing as mp
import time

def simple_test(queue):
    queue.put("Hello from subprocess")

queue = mp.Queue()
process = mp.Process(target=simple_test, args=(queue,))
process.start()
process.join(timeout=5)

if process.is_alive():
    print("Process hung!")
    process.terminate()
else:
    result = queue.get()
    print(f"Process worked: {result}")

Process worked: Hello from subprocess


In [18]:
# Test basic function
print("Testing basic function call...")
code = """
def foo(x: int) -> int:
    return x + x

x = 3
print(foo(x))
"""

try:
    result = python_repl_tool.invoke({"code": code, "timeout": 5})
    print(f"Success: {result}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Testing basic function call...
Success: ✗ Execution failed.
Error (QueueEmptyError): Result queue was empty.

Traceback:

Extra Context:
{}


In [14]:
# Test Fibonacci
from sparq.tools.python_repl.python_repl_tool import python_repl_tool

print("Testing Recursion with Fibonacci...")
code = """
def fib(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

print(f"fib(10): {fib(10)}")
"""

try:
    result = python_repl_tool.invoke({"code": code, "timeout": 30})
    print(f"Success: {result}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Testing Recursion with Fibonacci...
Success: ✗ Execution failed.
Error (QueueEmptyError): Result queue was empty.

Traceback:

Extra Context:
{}


In [15]:
# Test Fibonacci
from sparq.tools.python_repl.python_repl_tool import python_repl_tool

print("Testing Recursion with Fibonacci...")
code = """
def fib(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

print(f"fib(10): {fib(10)}")
"""

try:
    result = python_repl_tool.invoke({"code": code, "timeout": 30})
    print(f"Success: {result}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Testing Recursion with Fibonacci...
Success: ✗ Execution failed.
Error (QueueEmptyError): Result queue was empty.

Traceback:

Extra Context:
{}


In [16]:
# Test NumPy with detailed output
from sparq.tools.python_repl.python_repl_tool import python_repl_tool

print("Testing NumPy execution directly...")
code = """
import numpy as np
arr = np.array([1, 2, 3, 4, 5])
mean = arr.mean()
std = arr.std()
print(f"Mean: {mean}, Std: {std}")
mean
"""

try:
    result = python_repl_tool.invoke({"code": code, "timeout": 30})
    print(f"Success: {result}")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Testing NumPy execution directly...
Success: ✗ Execution failed.
Error (QueueEmptyError): Result queue was empty.

Traceback:

Extra Context:
{}


In [17]:
"""
Example: Using python_repl_tool with create_react_agent

This example demonstrates how to create a ReAct agent that can execute Python code
to solve mathematical problems and perform data analysis tasks.
"""

from pathlib import Path
import os

from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

from sparq.tools.python_repl.python_repl_tool import python_repl_tool
from sparq.utils.helpers import get_llm
from sparq.settings import Settings


def main():
    """
    Main function demonstrating python_repl_tool usage with create_react_agent.
    
    Examples:
        - Basic arithmetic
        - Data analysis with libraries
        - Multi-step calculations
    """
    # Setup environment and LLM
    settings = Settings()
    settings._load_env_variables(Path(".env"))
    
    if not os.getenv("GEMINI_API_KEY"):
        raise ValueError("GEMINI_API_KEY is not set in the environment variables.")
    
    llm = get_llm(model="gemini-2.0-flash", provider="google_genai")
    
    # Create ReAct agent with python_repl_tool
    agent = create_react_agent(
        model=llm,
        tools=[python_repl_tool]
    )
    
    # Example 1: Basic arithmetic
    print("=== Example 1: Basic Arithmetic ===")
    result = agent.invoke({
        "messages": [HumanMessage(content="Use Python to calculate 3.9345 + 5.4345")]
    })
    print(f"Final answer: {result['messages'][-1].content}\n")
    
    # Example 2: Multi-step calculation
    print("=== Example 2: Multi-Step Calculation ===")
    result = agent.invoke({
        "messages": [HumanMessage(
            content="Calculate the average of these numbers: 12, 45, 67, 23, 89, 34"
        )]
    })
    print(f"Final answer: {result['messages'][-1].content}\n")
    
    # Example 3: Data analysis with numpy
    print("=== Example 3: Using NumPy ===")
    result = agent.invoke({
        "messages": [HumanMessage(
            content="Use numpy to create an array [1, 2, 3, 4, 5] and calculate its mean and standard deviation"
        )]
    })
    print(f"Final answer: {result['messages'][-1].content}\n")
    
    # Example 4: Accessing tool call details
    print("=== Example 4: Inspecting Tool Calls ===")
    result = agent.invoke({
        "messages": [HumanMessage(content="Calculate 10 factorial using Python")]
    })
    
    # Find the AIMessage with tool calls
    for message in result["messages"]:
        if hasattr(message, "tool_calls") and message.tool_calls:
            for tool_call in message.tool_calls:
                print(f"Tool: {tool_call['name']}")
                print(f"Arguments: {tool_call['args']}")
                print(f"Code executed: {tool_call['args']['code']}")
                break
    
    print(f"\nFinal answer: {result['messages'][-1].content}\n")
    
    # Example 5: Handling errors gracefully
    print("=== Example 5: Error Handling ===")
    result = agent.invoke({
        "messages": [HumanMessage(
            content="Try to divide 10 by 0 in Python"
        )]
    })
    print(f"Agent response: {result['messages'][-1].content}\n")


def stream_example():
    """
    Example showing how to stream agent responses for real-time feedback.
    
    Note:
        Streaming is useful for long-running calculations or when you want
        to show progress to users.
    """
    settings = Settings()
    settings._load_env_variables(Path(".env"))
    
    llm = get_llm(model="gemini-2.0-flash", provider="google_genai")
    agent = create_react_agent(model=llm, tools=[python_repl_tool])
    
    print("=== Streaming Example ===")
    prompt = "Calculate the sum of squares from 1 to 100 using Python"
    
    for chunk in agent.stream(
        {"messages": [HumanMessage(content=prompt)]},
        config={"recursion_limit": 10}
    ):
        # Print each step as it happens
        for node_name, node_output in chunk.items():
            if "messages" in node_output:
                for msg in node_output["messages"]:
                    msg_type = type(msg).__name__
                    print(f"\n[{node_name}] {msg_type}")
                    if hasattr(msg, "content") and msg.content:
                        print(f"  Content: {msg.content[:200]}")
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"  Tool calls: {msg.tool_calls[0]['name']}")


def extract_code_from_tool_call(agent_output: dict) -> str:
    """
    Helper function to extract the Python code from tool calls in agent output.
    
    Args:
        agent_output: Output dictionary from agent.invoke()
        
    Returns:
        The Python code that was executed, or empty string if no tool calls found
    """
    for message in agent_output["messages"]:
        if hasattr(message, "tool_calls") and message.tool_calls:
            for tool_call in message.tool_calls:
                if tool_call["name"] == "python_repl_tool":
                    return tool_call["args"].get("code", "")
    return ""


if __name__ == "__main__":
    # Run basic examples
    main()
    
    # Run streaming example
    print("\n" + "="*60 + "\n")
    stream_example()
    
    # Example of extracting code
    print("\n" + "="*60 + "\n")
    print("=== Code Extraction Example ===")
    
    settings = Settings()
    settings._load_env_variables(Path(".env"))
    llm = get_llm(model="gemini-2.0-flash", provider="google_genai")
    agent = create_react_agent(model=llm, tools=[python_repl_tool])
    
    result = agent.invoke({
        "messages": [HumanMessage(content="Calculate 15 * 23 using Python")]
    })
    
    code = extract_code_from_tool_call(result)
    print(f"Code executed by agent: {code}")
    print(f"Final answer: {result['messages'][-1].content}")

=== Example 1: Basic Arithmetic ===
Final answer: 3.  9345 + 5.4345 = 9.369

=== Example 2: Multi-Step Calculation ===
Final answer: The average of the numbers is 45.0.

=== Example 3: Using NumPy ===


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT